In [ ]:
import os
from PIL import Image
import numpy as np
import sys
from collections import defaultdict
import cv2
import math

FEATURE = ((1, 2), (2, 1), (1, 3), (3, 1), (2, 2))

In [ ]:
def calcIntegral(im):
    ii = np.zeros((im.shape[0] + 1, im.shape[1] + 1))
    for x in range(im.shape[1]):
        for y in range(im.shape[0]):
            pixel_val = im[y, x]
            sumr[y, x] = sumr[y-1, x] + pixel_val
            ii[y+1, x+1] = ii[y+1, x] + sumr[y, x]
    
    return ii
    
def get_sum(iimg, tl,height,width):
    height = int(height)
    width = int(width)
    try:
        sum = iimg[tl[1]][tl[0]] - iimg[tl[1] + height][tl[0]] - iimg[tl[1]][tl[0] + width] + iimg[tl[1] + height][tl[0] + width]
    except:
        print(tl)
    return sum


def get_feature_list1(img_width, img_height):
    tot_list = []
    max_w, max_h = 8, 8
    for i in range(5):
        f_cnt = 0
        for w in range(FEATURE[i][0], max_w + 1, FEATURE[i][0]):
            for h in range(FEATURE[i][1], max_h + 1, FEATURE[i][1]):
                x = 0
                while(x < img_width - w + 1):
                    y = 0
                    while(y < img_height - h + 1):
                        a = RectangleRegion(FEATURE[i], (x, y), w, h, 1)
                        b = RectangleRegion(FEATURE[i], (x, y), w, h, -1)
                        tot_list.extend([a, b])
                        y += 1
                        f_cnt += 1  
                    x += 1
    print(len(tot_list))
    return tot_list


In [ ]:
class RectangleRegion(object):
    def __init__(self, f_type, pos, width, height, parity):
        self.alpha = 0.0
        self.type = f_type
        self.parity = parity
        self.t_left = pos
        self.width = width
        self.height = height

     
    def get_score(self, iimg):
        diff = 0
        if self.type == (1, 2):
            pos = get_sum(iimg, self.t_left, self.height / 2, self.width)
            neg = get_sum(iimg, (self.t_left[0], int(self.t_left[1] + self.height / 2)),  self.height / 2, self.width)
            diff = pos - neg

        elif self.type == (2, 1):
            pos = get_sum(iimg, self.t_left, self.height , int(self.width / 2))
            neg = get_sum(iimg, (int(self.t_left[0] + self.width / 2), self.t_left[1]), self.height , int(self.width / 2))
            diff = pos - neg

        elif self.type == (1, 3):
            pos = get_sum(iimg, self.t_left,  int(self.height / 3) , self.width) 
            pos += get_sum(iimg, (self.t_left[0], int(self.t_left[1] + 2 * self.height / 3)),  int(self.height / 3) , self.width)
            neg = get_sum(iimg, (self.t_left[0], int(self.t_left[1] + self.height / 3)),  int(self.height / 3) , self.width)
            diff = pos - neg 

        elif self.type == (3, 1):
            pos = get_sum(iimg, self.t_left, self.height, int(self.width/3))
            pos += get_sum(iimg, (int(self.t_left[0] + 2 * self.width / 3), self.t_left[1]), self.height, int(self.width/3))
            neg = get_sum(iimg, (int(self.t_left[0] + self.width / 3), self.t_left[1]), self.height, int(self.width/3))
            diff = pos - neg

        elif self.type == (2, 2):
            pos = get_sum(iimg, self.t_left, int(self.height / 2) , int(self.width / 2))
            neg = get_sum(iimg, (int(self.t_left[0] + self.width / 2), self.t_left[1]), int(self.height / 2) , int(self.width / 2))
            neg += get_sum(iimg, (self.t_left[0], int(self.t_left[1] + self.height / 2)), int(self.height / 2) , int(self.width / 2))
            pos += get_sum(iimg, (int(self.t_left[0] + self.width / 2), int(self.t_left[1] + self.height / 2)), int(self.height / 2) , int(self.width / 2))
            
        return pos - neg
        

    def feature_classifier(self, im):
        diff = self.get_score(im)
        a = self.parity * diff
        b = self.parity * 0
        if a < b:
            return 1
        else:
            return 0


In [ ]:
def classifier(classifiers,image):
    sum1 = 0.0
    sum2 = 0.0
    for feature in classifiers:
        sum1 += feature.feature_classifier(image) * feature.alpha

    for feature in classifiers:
        sum2 += 0.5*feature.alpha
    if sum1 >= sum2:
        return 1
    return 0



def find_acc(classifiers, test_poses_integral,test_negs_integral, skip_img = None):
    test_integral = test_poses_integral + test_negs_integral
    accuracy, fp, fn, tn, tp = (0.0 for i in range(5))
    idx = 0
    pos_cnt = len(test_poses_integral)
    neg_cnt = len(test_negs_integral)
    for idx in range(pos_cnt):
        guess = classifier(classifiers, test_poses_integral[idx])
        if guess == 1:
            tp += 1
            accuracy += 1
        if guess == 0:
            fn += 1

    for idx in range(neg_cnt):
        guess = classifier(classifiers,test_negs_integral[idx])
        if guess == 0:
            tn += 1
            accuracy += 1
        if guess == 1:
            fp += 1
    return accuracy, fp, fn, tp, tn

In [ ]:
def get_guessmatrix(img_integrals,features):
    guess = {}
    num_img = len(img_integrals)
    num_features = len(features)
    for i in range(num_img):
        image = img_integrals[i]
        for j in range(num_features):
            guess[(i,j)] = features[j].feature_classifier(image)
            
    return guess



In [ ]:
class Adaboost():
    def __init__(self,pos_integral, neg_integral, classi_num):
        self.pos_integral = pos_integral 
        self.pos_img_num = len(pos_integral)
        self.neg_img_num = len(neg_integral)
        self.neg_integral = neg_integral 
        self.classi_num = classi_num 
        self.classifiers = []
        self.img_integrals = pos_integral + self.neg_integral
        ground_truth_temp = [1] * self.pos_img_num + [0] * self.neg_img_num
        self.ground_truth = np.array(ground_truth_temp)
        pos_weights_temp = [1.0 / 2.0 * self.pos_img_num] * self.pos_img_num
        neg_weights_temp = [1.0 / 2.0 * self.neg_img_num] * self.neg_img_num
        self.weights = np.array(pos_weights_temp + neg_weights_temp)
        img_height, img_width = self.pos_integral[0].shape
        self.features = get_feature_list1(img_height - 1, img_width - 1)        
        self.guess = get_guessmatrix(self.img_integrals,self.features)
        
    def learn(self):
        selected_idxes=set()
        for t in range(self.classi_num):
            self.weights /= np.sum(self.weights)
            low_err = float('inf')
            selected_feature = None
            index = 0

            for i in range(len(self.features)):
                if i in selected_idxes:
                    continue
                feature = self.features[i]
                err = 1e-8
                for j in range(len(self.img_integrals)):
                    delta = self.weights[j] * abs(self.guess[(j,i)] - self.ground_truth[j])
                    err += delta
                if err < low_err:
                    low_err = err
                    selected_feature = feature
                    index = i
            training_accuracy = 0.0
            power = 0.0
            for j in range(len(self.img_integrals)):
                if self.guess[(j,index)] == self.ground_truth[j]:
                    power = 0.0
                    training_accuracy += 1
                else:
                    power = 1.0
                self.weights[j] = self.weights[j] * (low_err / (1.0 - low_err)) ** power

            selected_idxes.add(index)
            # selected_feature.training_accuracy = training_accuracy / (len(self.img_integrals))
            selected_feature.alpha = math.log((1.0 - low_err) / low_err)
            self.classifiers.append(selected_feature)
            #print('round: %d' % t)
        return self.classifiers


In [ ]:
pos_training_path = '/content/drive/MyDrive/dataset/trainset/faces'
neg_training_path = '/content/drive/MyDrive/dataset/trainset/non-faces'
pos_testing_path = '/content/drive/MyDrive/dataset/testset/faces'
neg_testing_path = '/content/drive/MyDrive/dataset/testset/non-faces'

allpaths = [pos_training_path,neg_training_path,pos_testing_path,neg_testing_path]
allimages = {}
for path in allpaths:
    images = []
    for im_name in os.listdir(path):
        if im_name.lower().endswith('.png'):
            im = cv2.cvtColor(cv2.imread(os.path.join(path,im_name)) , cv2.COLOR_BGR2GRAY)
            im = np.asarray(im,dtype=np.float64)
            im = im/np.max(im)
            images.append(calcIntegral(im))
    allimages[path] = images

train_poses_integral = allimages[pos_training_path]
train_negs_integral = allimages[neg_training_path]
test_poses_integral = allimages[pos_testing_path]
test_negs_integral = allimages[neg_testing_path]

classi_num = 10
adabo = Adaboost(train_poses_integral, train_negs_integral, classi_num)
classifiers = adabo.learn()
accuracy, fp, fn, tp, tn= find_acc(classifiers, test_poses_integral, test_negs_integral)

print('Total accuracy: %f (%d/%d)' % (accuracy / len(test_poses_integral + test_negs_integral), accuracy,
                                        len(test_poses_integral + test_negs_integral)))
print('False Positive: %f (%d/%d)' % (fp / len(test_negs_integral), fp, len(test_negs_integral)))
print('False Negative: %f (%d/%d)\n' % (fn / len(test_poses_integral), fn, len(test_poses_integral)))

52336
Total accuracy: 0.642857 (18/28)
False Positive: 0.428571 (6/14)
False Negative: 0.285714 (4/14)

